## Scrape the data first

In [ ]:
# Import libraries
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from geopandas_view import view 

In [ ]:
driver = webdriver.Chrome('C:\Windows\chromedriver.exe')

In [ ]:
prices = []
beds = []
baths = []
sizes = []
addresses = []
driver.get('https://www.realtor.com/realestateandhomes-search/New-York_NY')

In [ ]:
content = driver.page_source
soup = BeautifulSoup(content, features='html.parser')

for element in soup.findAll('li', attrs={'class':'component_property-card'}):
    price = element.find('span', attrs={'data-label': 'pc-price'})
    bed = element.find('li', attrs={'data-label': 'pc-meta-beds'})
    bath = element.find('li', attrs={'data-label': 'pc-meta-baths'})
    size = element.find('li', attrs={'data-label': 'pc-meta-sqft'})
    address = element.find('div', attrs={'data-label': 'pc-address'})
    
    if bed and bath:
        nr_beds = bed.find('span', attrs={'data-label': 'meta-value'})
        nr_baths = bath.find('span', attrs={'data-label': 'meta-value'})
        
        #pre processing
        #nr_beds.text = nr_beds.text.replace("+","")
        #nr_baths.text = nr_baths.text.replace("+","")
        
        if nr_beds and float(nr_beds.text.replace("+","")) >= 2 and nr_baths and float(nr_baths.text.replace("+","")) >= 1:
            # if nr_beds == (nr_beds + '+'):
            #     print(float())
            beds.append(nr_beds.text)
            baths.append(nr_baths.text)
            
            if price and price.text:
                prices.append(price.text)
            else:
                prices.append('No display data')
            if size and size.text:
                sizes.append(size.text)
            else:
                sizes.append('No display data')

            if address and address.text:
                addresses.append(address.text)
            else:
                addresses.append('No display data')

In [ ]:
df = pd.DataFrame({'Address': addresses, 'Price': prices, 'Beds': beds, 'Baths': baths, 'Sizes': sizes})
df.to_csv('listings.csv', index=False, encoding='utf-8')

In [ ]:
# Second method of using online web scraping tool
# Create a free web scraping API account first
# It is optional

## Geocode the scraped data 

In [ ]:
import pandas as pd
import requests
import json
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# read restaurants csv
df = pd.read_csv("listings.csv")
df

In [ ]:
len(df)

In [ ]:
# Import geocoding tool
from geopandas.tools import geocode

# Geocode addresses using Nominatim. Remember to provide a custom "application name" in the user_agent parameter!
geo = geocode(df['Address'], provider='nominatim', user_agent='autogis_ps', timeout=3)

In [ ]:
filtered_geo = gpd.GeoDataFrame(geo).where(lambda x : x!=None).dropna()
filtered_geo

In [ ]:
view(filtered_geo)